In [5]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from matplotlib import pyplot as plt
import scipy.stats as sts

macro = np.loadtxt("/Users/benjaminlim/Documents/BootCamp2017/Labs/Econ/Week 4/MacroSeries.txt", delimiter = ",")
c = macro[:, 0]
w = macro[:, 2]
k = macro[:, 1]
r = macro[:, 3]

In [6]:
'''
    moment1 = np.zeros((len(z),1))
    moment2 = np.zeros((len(z),1))
    moment3 = np.zeros((len(z),1))
    moment4 = np.zeros((len(z),1))
    return moment1, moment2, moment3, moment4


'''

def data_moments():
   
    datamom1 = 0
    datamom2 = 0
    datamom3 = 0
    datamom4 = 0
    
    return datamom1 ,datamom2 ,datamom3 , datamom4 


def model_moments(r, c, k, w, alpha, beta, rho, mu):
    denom = alpha * (k ** (alpha - 1))
    z = np.log(r / denom) 
    znext = z[1:]
    z = z[:-1]
    
    cnext = c[1:]
    c = c[:-1]
    knext = k[1:]
    k = k[:-1]
    wnext = w[1:]
    w = w[:-1]
    
    modelmom1 = np.mean(znext - rho*z - (1-rho)*mu)
    modelmom2 = np.mean((znext - rho*z - (1-rho)*mu)*z)
    modelmom3 = np.mean(beta*alpha*np.exp(znext)*(knext**(alpha-1))*c/cnext - 1)
    modelmom4 = np.mean((beta*alpha*np.exp(znext)*(knext**(alpha-1))*c/cnext - 1)*w)
    
    return modelmom1, modelmom2, modelmom3, modelmom4

def err_vec(r, c, k, w, alpha, beta, rho, mu, simple):
    '''
    --------------------------------------------------------------------
    This function computes the vector of moment errors (in percent
    deviation from the data moment vector) for GMM.
    --------------------------------------------------------------------
    INPUTS:
    xvals  = (N,) vector, test scores data
    mu     = scalar, mean of the normally distributed random variable
    sigma  = scalar > 0, standard deviation of the normally distributed
             random variable
    cutoff = scalar or string, ='None' if no cutoff is given, otherwise
             is scalar upper bound value of distribution. Values above
             this value have zero probability
    simple = boolean, =True if errors are simple difference, =False if
             errors are percent deviation from data moments
    
    OTHER FUNCTIONS AND FILES CALLED BY THIS FUNCTION:
        data_moments()
        model_moments()
    
    OBJECTS CREATED WITHIN FUNCTION:
    mean_data  = scalar, mean value of data
    var_data   = scalar > 0, variance of data
    moms_data  = (2, 1) matrix, column vector of two data moments
    mean_model = scalar, mean value from model
    var_model  = scalar > 0, variance from model
    moms_model = (2, 1) matrix, column vector of two model moments
    err_vec    = (2, 1) matrix, column vector of two moment error
                 functions
    
    FILES CREATED BY THIS FUNCTION: None
    
    RETURNS: err_vec
    --------------------------------------------------------------------
    '''
    datamom1, datamom2, datamom3, datamom4 = data_moments()
    moms_data = np.array([[datamom1], [datamom2], [datamom3], [datamom4]])
    modelmom1, modelmom2, modelmom3, modelmom4 = model_moments(r, c, k, w, alpha, beta, rho, mu)
    moms_model = np.array([[modelmom1], [modelmom2], [modelmom3], [modelmom4]])
    if simple:
        err_vec = moms_model - moms_data
    else:
        err_vec = (moms_model - moms_data) / (moms_data)
    
    return err_vec


def criterion(params, *args):
    '''
    --------------------------------------------------------------------
    This function computes the GMM weighted sum of squared moment errors
    criterion function value given parameter values and an estimate of
    the weighting matrix.
    --------------------------------------------------------------------
    INPUTS:
    params = (2,) vector, ([mu, sigma])
    mu     = scalar, mean of the normally distributed random variable
    sigma  = scalar > 0, standard deviation of the normally distributed
             random variable
    args   = length 3 tuple, (xvals, cutoff, W_hat)
    xvals  = (N,) vector, values of the truncated normally distributed
             random variable
    cutoff = scalar or string, ='None' if no cutoff is given, otherwise
             is scalar upper bound value of distribution. Values above
             this value have zero probability
    W_hat  = (R, R) matrix, estimate of optimal weighting matrix
    
    OTHER FUNCTIONS AND FILES CALLED BY THIS FUNCTION:
        norm_pdf()
    
    OBJECTS CREATED WITHIN FUNCTION:
    err        = (2, 1) matrix, column vector of two moment error
                 functions
    crit_val   = scalar > 0, GMM criterion function value
    
    FILES CREATED BY THIS FUNCTION: None
    
    RETURNS: crit_val
    --------------------------------------------------------------------
    '''
    alpha, beta, rho, mu = params
    r, c, k, w, W = args
    err = err_vec(r, c, k, w, alpha, beta, rho, mu, simple = True)
    crit_val = err.T @ W @ err
    
    return crit_val

In [25]:
import scipy.optimize as opt

alpha_init = 0.5
beta_init = 0.5
rho_init = 0.2
mu_init = 2

params_init = np.array([alpha_init, beta_init, rho_init, mu_init])
W = np.eye(4)
gmm_args = (r, c, k, w, W)
results = opt.minimize(criterion, params_init, args=(gmm_args),
                       method='L-BFGS-B', bounds=((1e-10, 1-1e-10),(1e-10, 1-1e-10),
                               (-1+1e-10, 1-1e-10), (1e-10, None)))
alpha_GMM1, beta_GMM1, rho_GMM1, mu_GMM1 = results.x
print('alpha_GMM1=', alpha_GMM1, ' beta_GMM1=', beta_GMM1,' rho_GMM1=', rho_GMM1, ' mu_GMM1=', mu_GMM1)

alpha_GMM1= 0.500000000121  beta_GMM1= 0.989999995126  rho_GMM1= 0.200000000024  mu_GMM1= 2.0


In [8]:
datamom1, datamom2, datamom3, datamom4 = data_moments()
modelmom1, modelmom2, modelmom3, modelmom4 = model_moments(r, c, k, w, alpha_GMM1, beta_GMM1, rho_GMM1, mu_GMM1)
err1 = err_vec(r, c, k, w, alpha_GMM1, beta_GMM1, rho_GMM1, mu_GMM1, simple = True).reshape(4,)
print('datamom1 =', datamom1, ', datamom2 =', datamom2, ', datamom3 =', datamom3,', datamom4 =', datamom4)
print('modelmom1 =', modelmom1, ', modelmom2 =', modelmom2, ', modelmom3 =', modelmom3,', modelmom4 =', modelmom4)
print('Error vector=', err1)
results

datamom1 = 0 , datamom2 = 0 , datamom3 = 0 , datamom4 = 0
modelmom1 = 0.00097130351417 , modelmom2 = 0.00353188251713 , modelmom3 = -4.8495692501e-09 , modelmom4 = -0.0505104828882
Error vector= [  9.71303514e-04   3.53188252e-03  -4.84956925e-09  -5.05104829e-02]


      fun: array([[ 0.00256473]])
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -1.44945373e-01,   4.40361035e+04,  -7.09349246e-05,
        -9.00515066e-03])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 190
      nit: 24
   status: 0
  success: True
        x: array([  9.41287589e-01,   9.89999995e-01,   1.83358671e-04,
         9.99815319e-01])